In [2]:
import re
import os
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter

from konlpy.tag import Okt
okt = Okt()

#비속어사전
from urllib.request import urlopen
import json
url = "https://raw.githubusercontent.com/organization/Gentleman/master/resources/badwords.json"
response = urlopen(url)
data_json = json.loads(response.read())  

In [55]:
#영어,숫자,특수문자,한글자 단어 제거
def text_preprocessor(s):
    pattern = r'[^가-힣]'
    s = re.sub(pattern=pattern, repl=' ', string=s)
    
    #표제어추출
    s_split = okt.morphs(s, stem=True)
      
    s_list = []
    for word in s_split:
        if len(word) !=1:
            s_list.append(word)
            
    return s_list

In [56]:
kmbti = ['인프젠데',
 '엔프젠데',
 '엣팁입니',
 '엔프피특',
 '잉뿌삐',
 '엣프제',
 '엔프핀데',
 '엣티제',
 '엣프피',
 '엔트제',
 '엣프핀데',
 '티젠데',
 '인프피님들',
 '인티',
 '찐인프피',
 '유미',
 '엣프',
 '잇프피님들',
 '엔뿌삐',
 '엣팁인데',
 '엣팁',
 '엣티젠데',
 '엔프제',
 '잉뿌삐들',
 '엔프피',
 '인프피특',
 '잇프피인줄',
 '인팁',
 '인트제',
 '엣팁들',
 '인프',
 '프피',
 '잇티제',
 '엡팁',
 '인프피',
 '인프핀데',
 '앤팁',
 '잇프피',
 '엣티제들',
 '인티젠데',
 '엣티',
 '잇프피들',
 '찐프피',
 '엔쁘삐',
 '인프피들',
 '강유미',
 '엠비티아이',
 '엣프피들',
 '엥뿌삐',
 '인뿌삐',
 '잇프',
 '잇프젠데',
 '엥뿌삐들',
 '엔프피들',
 '인티제',
 '프제',
 '인프피로',
 '잇프핀데',
 '엣프젠데',
 '티제',
 '잇프피입니',
 '엔프']

In [57]:
def words_tokonizer(text):
    words = []    
    s_list = text_preprocessor(text)
    norm_txt = okt.normalize(' '.join(s_list))
    pos_lst = okt.pos(norm_txt)
    
    
    #접속사, 조사, ㅋㅋ,  비속어, 엣푸피(kmbti) 아니라면 제거.
    for word, pos in pos_lst:
        #print(word, pos)
        # & (word not in kmbti)
        if (pos != 'Conjunction') & (pos != 'KoreanParticle') & (pos != 'Josa') & (word not in data_json['badwords']) & (word not in kmbti) & (len(word) > 1) :
            words.append(word)
    return words

In [59]:
#전처리_1 -> text_preprocessor, words_tokonizer, 12글자 이하제거

for file in os.listdir('MBTItotal'):
    file_name = file.split('_')[0]
    df = pd.read_csv(f'MBTItotal/{file}', header=None)
    df.columns = ['index', 'id', 'text']
    df.drop(['index', 'id'], axis=1, inplace=True)
    df.dropna(inplace=True) #결측치(null) 제거
    
    s_lst = []
    text_lst = []
    for row in tqdm(df.iterrows(), total=df.shape[0]):
        s_lst += words_tokonizer(row[1]['text']) #row[1]['text'] 문자열
        
        k = ' '.join(words_tokonizer(row[1]['text']))
        if len(k)>12 : #12글자 이하 제거
            text_lst.append(k)
    
    df1 = pd.DataFrame(zip(text_lst))
    df1.columns = ['new_text']


    df1.to_csv(f"{file_name}.csv", encoding='utf-8-sig', index=True)
    
    #단어 빈도수(이 파일 이용해서 빈도수 5이하인 단어 전체 댓글에서 모두제거시 사용.)
    '''
    count = Counter([s for s in s_lst])
    word_freq = pd.DataFrame(count.items(), columns=['word', 'frequency']).sort_values(by='frequency', ascending=False)
    word_freq.to_csv(f"{file_name}_word_freq.csv", encoding='utf-8-sig', index=False)'''

  0%|          | 0/4023 [00:00<?, ?it/s]

  0%|          | 0/4350 [00:00<?, ?it/s]

  0%|          | 0/4328 [00:00<?, ?it/s]

  0%|          | 0/5478 [00:00<?, ?it/s]

  0%|          | 0/4110 [00:00<?, ?it/s]

...

  0%|          | 0/6362 [00:00<?, ?it/s]

  0%|          | 0/5048 [00:00<?, ?it/s]

  0%|          | 0/6004 [00:00<?, ?it/s]

  0%|          | 0/4402 [00:00<?, ?it/s]

  0%|          | 0/4624 [00:00<?, ?it/s]

In [14]:
#전처리_2 -> 빈도수 5이하인 단어, 댓글에서 모두제거

import sys
sys.stdout = open('output.txt','w')

file_lst = ['enfj_un_word', 'enfp_un_word','entj_un_word','entp_un_word','esfj_un_word','esfp_un_word', 
            'estj_un_word', 'estp_un_word','infj_un_word','infp_un_word','intj_un_word','intp_un_word',
            'isfj_un_word', 'isfp_un_word','istj_un_word','istp_un_word'] #빈도수 5이하인 단어 모아놓은 파일들.

for file in os.listdir('MBTI1total'):#1차 전처리 완료된 16개 파일
    #file_name = file[:4]+'TT'#나중에저장될 최종파일
    
    df = pd.read_csv(f'MBTI1total/{file}', header=None)
    
    total_lst = []
    
    #un_word contact
    contact=file[:4].lower() #해당mbti문자열
    print(contact)
    
    for tt in file_lst : 
        if contact in tt : 
            yesmy = tt #yesmy : isfj_un_word
    #print(yesmy)
    
    file1 = f'{yesmy}.csv'
    #print(file1)
    df2 = pd.read_csv(file1) #un_word 파일 불러오기 -> df2['un_word']
    #print(df2)
    #print("----")
    
    target = df2['un_word'].values.tolist() #비교할 un_word리스트***
    #print(target)
    
    for index, row in df.iterrows():
        kt = [] #un_word가 없는 하나의 문장.
        
        n = row[1].split() 
        #print(n)
        
        for lk in n :
            if lk not in target : 
                kt.append(lk)
        
        total_lst.append(kt)
        #print(kt)
                
    print(total_lst)